In [12]:
import os
import numpy as np
from pyciemss.PetriNetODE.interfaces import load_and_sample_petri_model

In [25]:
MIRA_PATH = "test/models/april_ensemble_demo/"

filename = "BIOMD0000000955_template_model.json"
filename = os.path.join(MIRA_PATH, filename)

num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]
start_state = {}

samples = load_and_sample_petri_model(
    filename, num_samples, timepoints=timepoints, add_uncertainty=True
)

output_path = "notebook/integration_demo/results_petri/"

for name, sample in samples.items():
    # Leaving this as simple as possible, but we're very happy to add more meta-information to the
    # output files if you need it. We can also push this into `load_and_sample_petri_model`.
    # Also, very happy to squash some of these into a `params` and `states` files rather than one file per
    # parameter/state.
    if sample.ndim == 1:
        # Any 1D array is a sample from the distribution over parameters.
        # Any 2D array is a sample from the distribution over states. ()
        name = name + "_param"

    np.savetxt(os.path.join(output_path, f"{name}.csv"), sample.data.numpy())

In [29]:
samples

{'beta': tensor([0.0118, 0.0114, 0.0118, 0.0117, 0.0119, 0.0099, 0.0103, 0.0120, 0.0102,
         0.0118, 0.0101, 0.0106, 0.0114, 0.0112, 0.0109, 0.0109, 0.0103, 0.0108,
         0.0118, 0.0114, 0.0121, 0.0104, 0.0102, 0.0102, 0.0117, 0.0116, 0.0112,
         0.0102, 0.0105, 0.0106, 0.0112, 0.0113, 0.0101, 0.0112, 0.0116, 0.0112,
         0.0110, 0.0117, 0.0111, 0.0118, 0.0114, 0.0112, 0.0113, 0.0102, 0.0107,
         0.0103, 0.0115, 0.0120, 0.0118, 0.0111, 0.0111, 0.0114, 0.0113, 0.0108,
         0.0099, 0.0107, 0.0119, 0.0105, 0.0111, 0.0110, 0.0119, 0.0118, 0.0105,
         0.0121, 0.0117, 0.0115, 0.0117, 0.0101, 0.0113, 0.0115, 0.0108, 0.0114,
         0.0109, 0.0107, 0.0120, 0.0107, 0.0118, 0.0116, 0.0121, 0.0114, 0.0119,
         0.0121, 0.0104, 0.0106, 0.0110, 0.0105, 0.0112, 0.0107, 0.0106, 0.0105,
         0.0099, 0.0115, 0.0114, 0.0105, 0.0108, 0.0110, 0.0119, 0.0111, 0.0113,
         0.0116]),
 'gamma': tensor([0.4934, 0.4383, 0.4920, 0.4158, 0.4250, 0.4340, 0.4815, 0.4706, 

In [28]:
# One value per sample
samples["beta"].shape

torch.Size([100])

In [31]:
# One value per sample per timepoint
samples["Ailing_sol"].shape

torch.Size([100, 5])